In [1]:
# Initialize libraries
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
from IPython.display import display as dspl
from matplotlib import patches
from operator import sub

# Increase pixel resolution
plt.rcParams["figure.dpi"] = 100

def calculate_head(hr, hl, L, x):
    """
    Calculate hydraulic head along x for 1D Darcy flow between two Dirichlet B.C. 
    
    Keyword Arguments:
    hl -- hydraulic head at x=0 in m; float
    hr -- hydraulic head at x=L in m; float
    L -- distance to hl in m; float
    x -- positions along x in m; float or np.ndarray
    
    Return:
    h -- hydraulic head at positions x; float or np.ndarray
    """
    h=(hl**2-(hl**2-hr**2)/L*x)**0.5
    return h


def get_aspect(ax):
    """
    From: https://stackoverflow.com/questions/41597177/get-aspect-ratio-of-axes
    Accessed: 08/05/2023
    """
    # Total figure size
    figW, figH = ax.get_figure().get_size_inches()
    # Axis size on figure
    _, _, w, h = ax.get_position().bounds
    # Ratio of display units
    disp_ratio = (figH * h) / (figW * w)
    # Ratio of data units
    # Negative over negative because of the order of subtraction
    data_ratio = sub(*ax.get_ylim()) / sub(*ax.get_xlim())

    return disp_ratio / data_ratio

# Length of x domain in m
domain_length = 100
# Elevation of land surface at x=0 in m above datum
land_elevation = 2
# Elevation of sea bed at x=domain_length in m above datum
sea_elevation = -3
# Evelvation where to cut the plot if the full extent is not shown
base_elevation = -4
# Proxy for how steep the slope from land surface to sea bed should be
proxy_slope = 10
# Depth of groundwater table from land surface at x=0 in m
gw_depth = 0.5

# Calculate landscape geometry
plot_x = np.append(np.arange(0, domain_length, 1), domain_length)
norm_x = np.linspace(-1*proxy_slope, 1*proxy_slope, len(plot_x))
plot_y = np.arctan(1/norm_x)
plot_y[norm_x < 0] = plot_y[norm_x < 0] + np.pi
scale = (land_elevation - sea_elevation) / (np.max(plot_y) - np.min(plot_y))
offset = sea_elevation - plot_y[-1] * scale
plot_y = plot_y * scale + offset

def plot_domain(sea_lvl, dens_fresh=1000, dens_salt=1025, full_extent=False):
    """
    Plot groundwater table, sea level and resuling fresh-salt water interface 
    according to Ghyben-Herzberg
    
    Keyword Arguments:
    sea_lvl -- sea level in m above datum; float
    dens_fresh -- density fresh water in kg/m^3; float
    dens_salt -- density salt water in kg/m^3; float
    full_extent -- show full y extend of fresh-salt water interface; bool
    
    Return:
    -
    """
    # Prepare figure and subplots
    fig = plt.figure(figsize=(7,5))
    ax = fig.add_subplot(1, 1, 1)

    # x position of intersect between groundwater table and sea level
    intersect = 1 / np.tan((sea_lvl-offset)/scale) * domain_length / (2*proxy_slope) + domain_length / 2

    # Calculate groundwater table with Dirichlet b.c. at left and right side and Dupuit assumption
    # Another assumption is made that the datum is the sea (i.e. head is 0 at intersection) 
    # This assumption is from Todd and Mays, 2005 but seems arbitary. Another datum affects the shape
    # of the groundwater table which is why it's more meant for illustrative purposes
    wt_x = np.append(np.arange(0,intersect, 0.5),intersect)
    # Flip calculation to prevent computational errors when x becomes L
    h = np.flip(calculate_head(land_elevation-sea_lvl-gw_depth, 0, intersect, wt_x))
    wt_y = h + sea_lvl
    
    # Calculate fresh-salt water interface according to Ghyben-Herzberg
    bnd_x = wt_x
    bnd_y = sea_lvl - dens_fresh / (dens_salt - dens_fresh) * h
    
    # Plot landscape
    plt.plot(plot_x, plot_y, c="brown")
    # Plot groundwater table
    plt.plot(wt_x,wt_y, c="grey")
    # Plot fresh-salt water interface
    plt.plot(bnd_x, bnd_y)
    # Plot sea level
    plt.hlines(sea_lvl, intersect, domain_length)
    
    if not full_extent:
        y_min = base_elevation
    else:
        y_min = np.min(bnd_y)
    
    # Fill the ocean
    plt.fill_between(x=np.append(bnd_x, domain_length),
                     y1=np.append(bnd_y, sea_lvl),
                     y2=np.repeat(y_min, len(bnd_y) + 1),
                     alpha=0.5)
    
    # Fill the land
    plt.fill_between(x=plot_x,
                     y1=plot_y,
                     y2=np.repeat(y_min, len(plot_x)),
                     alpha=0.5,
                     color="brown")
    
    # Define x and y limits
    ax.set_xlim(0, domain_length)

    ax.set_ylim(y_min, land_elevation*1.1)

    # Plot groundwater triangle
    ratio = get_aspect(ax)
    x_arrow = intersect / 2
    y_arrow = calculate_head(0, land_elevation-sea_lvl-gw_depth, intersect, x_arrow) + sea_lvl
    height_scale = 0.002 * domain_length
    width_scale = ratio * height_scale / np.sqrt(3)
    ax.add_patch(patches.Polygon([[x_arrow, y_arrow],
                                  [x_arrow-width_scale, y_arrow+height_scale],
                                  [x_arrow+width_scale, y_arrow+height_scale]],
                                  edgecolor="grey",
                                  facecolor="grey"))
    height_scale1 = (land_elevation - sea_elevation) * 0.012
    height_scale2 = height_scale1 * 2
    width_scale1 = width_scale
    width_scale2 = width_scale1 * 0.8
    ax.hlines(y= y_arrow-height_scale1, xmin=x_arrow-width_scale1, xmax=x_arrow+width_scale1, 
              colors="grey", linewidth=1.5) 
    ax.hlines(y= y_arrow-height_scale2, xmin=x_arrow-width_scale2, xmax=x_arrow+width_scale2, 
              colors="grey", linewidth=1.5) 
    
# Full Extent widget
full_extent_WT = widgets.Checkbox(
                        value=False,
                        description="Full Extent",
                        disabled=False
                    )

# Sea level widget
sea_lvl_WT = widgets.BoundedFloatText(value=(sea_elevation+land_elevation)/2, min=sea_elevation+0.5, 
                                      max=land_elevation-gw_depth-0.5, step=0.1, 
                                 description=r'$h_{Sea}, m$:', disabled=False,
                                 continuous_update=True)

# Density fresh water widget
dens_fresh_WT = widgets.BoundedFloatText(value=1000, min=995, max=1300, step=1, 
                                 description=r'$\rho_{fresh}, \frac{kg}{m^3}$:', disabled=False,
                                 continuous_update=True)

# Density salt water widget
dens_salt_WT = widgets.BoundedFloatText(value=1025, min=995, max=1300, step=1, 
                                 description=r'$\rho_{salt}, \frac{kg}{m^3}$:', disabled=False,
                                 continuous_update=True)
# Start interactive plot
interact(plot_domain,
         sea_lvl = sea_lvl_WT,
         dens_fresh = dens_fresh_WT,
         dens_salt = dens_salt_WT,
         full_extent = full_extent_WT)

/tmp/ipykernel_10143/2039127753.py:62: RuntimeWarning: divide by zero encountered in true_divide
  plot_y = np.arctan(1/norm_x)


interactive(children=(BoundedFloatText(value=-0.5, continuous_update=True, description='$h_{Sea}, m$:', max=1.…

<function __main__.plot_domain(sea_lvl, dens_fresh=1000, dens_salt=1025, full_extent=False)>